# Import necessary packages

In [ ]:
# pip install git+https://github.com/fspinna/borf.git@xai-improvements

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from sklearn.preprocessing import LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline

plt.style.use('default')

# Load single cell data

In [2]:
data = pd.read_csv("temporal_data_ready_normalized.csv", index_col=0)
data = data.dropna(axis=0)

columns = data.columns

genes = np.array([name.split("_")[0] for name in list(data.index)])
cells = np.array(["_".join(name.split("_")[1:]) for name in list(data.index)])

enc_genes = LabelEncoder()
enc_cells = LabelEncoder()

enc_genes.fit(genes)
enc_cells.fit(cells)

X = data.values[:, np.newaxis, :]
y_genes = enc_genes.transform(genes)
y_cells = enc_cells.transform(cells)

X.shape, y_genes.shape, y_cells.shape, genes.shape, cells.shape

((22000, 1, 6), (22000,), (22000,), (22000,), (22000,))

## BORF

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeClassifier
import numpy as np
from fast_borf import BorfBuilder
from fast_borf.pipeline.zero_columns_remover import ZeroColumnsRemover
from fast_borf.pipeline.reshaper import ReshapeTo2D
from fast_borf.pipeline.to_scipy import ToScipySparse
from fast_borf.xai.mapping import BagOfReceptiveFields

In [4]:
# Setup the BORF builder
builder = BorfBuilder(
    n_jobs=-2, window_size_min_window_size=2, alphabets_min_symbols=3, alphabets_max_symbols=4, min_window_to_signal_std_ratio=0.0,
    pipeline_objects=[
        (ReshapeTo2D, dict(keep_unraveled_index=True)),
        (ZeroColumnsRemover, dict(axis=0)),
        (ToScipySparse, dict()),
        ],
)
borf = builder.build(X)

In [6]:
X_transformed = borf.fit_transform(X)

In [7]:
mapper = BagOfReceptiveFields(borf)
mapper.build(X[:100])

/Users/francesco/miniforge3/envs/timeseries_dl/lib/python3.12/site-packages/numba/typed/typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to uint64. Precision may be lost.
  return d[key]


In [8]:
mapper.mapping

array([[ 0,  0,  0],
       [ 0,  0,  2],
       [ 0,  0,  4],
       [ 0,  0,  6],
       [ 1,  0,  0],
       [ 1,  0,  2],
       [ 1,  0,  4],
       [ 1,  0,  6],
       [ 2,  0,  0],
       [ 2,  0,  2],
       [ 2,  0,  4],
       [ 2,  0,  6],
       [ 3,  0,  0],
       [ 3,  0,  2],
       [ 3,  0,  5],
       [ 3,  0,  6],
       [ 3,  0,  7],
       [ 3,  0,  8],
       [ 3,  0, 11],
       [ 3,  0, 14],
       [ 3,  0, 15],
       [ 3,  0, 16],
       [ 3,  0, 17],
       [ 3,  0, 18],
       [ 3,  0, 19],
       [ 3,  0, 20],
       [ 3,  0, 21],
       [ 3,  0, 22],
       [ 3,  0, 23],
       [ 3,  0, 24],
       [ 3,  0, 25],
       [ 3,  0, 26],
       [ 3,  0, 29],
       [ 3,  0, 32],
       [ 3,  0, 33],
       [ 3,  0, 34],
       [ 3,  0, 35],
       [ 3,  0, 38],
       [ 3,  0, 40],
       [ 3,  0, 42],
       [ 3,  0, 45],
       [ 3,  0, 46],
       [ 3,  0, 47],
       [ 3,  0, 48],
       [ 3,  0, 51],
       [ 3,  0, 54],
       [ 3,  0, 55],
       [ 3,  

In [9]:
mapper.receptive_fields_[10].word_array

array([1, 1], dtype=int32)